In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

import warnings
warnings.filterwarnings("ignore")

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
%%capture
import torchvision
from torchvision import transforms
import torch.nn as nn
import torch
from torch.utils.data import DataLoader
from os import listdir
from PIL import ImageOps, Image
from glob import glob

In [0]:
# Fonction d'entrainement
def training(model, train_dataloader, valid_dataloader=None, epoch=5, learning_rate=0.1, use_gpu=False):
  
  criterion = nn.CrossEntropyLoss()
  optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
  
  train_accu = []
  train_losses = []
  
  if valid_dataloader:
    val_accu = []
    val_loss = []
    
  for i in range(epoch):
    model.train()
    
    print('Starting epoch number {} on {} ...'.format(i+1, epoch))
    
    true = []
    pred = []
    train_loss = []
    len_train = len(train_dataloader)
    for ind, batch in enumerate(train_dataloader):
      print('\rBatch : {}/{}'.format(ind+1, len_train), end='')
      x_train, y_train = batch
      
      # Aller le GPU, fais ton taff
      if use_gpu:
        inputs = inputs.cuda()
        targets = targets.cuda()
      
      optimizer.zero_grad()
        
      outputs = model(inputs)
      
      loss = criterion(outputs, targets)
      loss.backward()
      optimizer.step()
      
      predictions = outputs.max(dim=1)[1]
      
      train_loss.append(loss.item())
      true.extend(targets.data.cpu().numpy().tolist())
      pred.extend(predictions.data.cpu().numpy().tolist())
    
    accu_score = accuracy_score(true, pred) * 100
    loss_score = sum(train_loss) / len(train_loss)
    
    print('\nTrain score : Accuracy = {:.2f} - Loss = {:.2f}'.format(accu_score, loss_score), end='')
    
    
    train_accu.append(accuracy_score(true, pred) * 100)
    train_losses.append(loss_score)
      
    if valid_dataloader:
      vaccu, vloss = validating(model, valid_dataloader, use_gpu)
      print(' | Validation score : Accuracy = {:.2f} - Loss = {:.2f}'.format(vaccu, vloss))
      val_accu.append(vaccu)
      val_loss.append(vloss)
    else:
      print()
  
  if valid_dataloader:
    return train_accu, train_losses, val_accu, val_loss
  else:
    return train_accu, train_loss
  
  return None, None
  

# Fonction de validation
def validating(model, dataloader, use_gpu=False):
  true =[]
  pred = []
  val_loss = []

  criterion = nn.NLLLoss()
  model.eval()

  for batch in dataloader:

    x_valid, y_valid = batch

    # On envoit les données au GPU pour le traitement
    if use_gpu:
      inputs = inputs.cuda()
      targets = targets.cuda()
    
    outputs = model(inputs)

    predictions = outputs.max(dim=1)[1]

    val_loss.append(criterion(outputs, targets).item())
    true.extend(targets.data.cpu().numpy().tolist())
    pred.extend(predictions.data.cpu().numpy().tolist())
  
  accu_score = accuracy_score(true, pred) * 100
  loss_score = sum(val_loss) / len(val_loss)
  return accu_score, loss_score


# Fonction de prédiction
#def predict(model, sentence, vocab, max_len):
#  tokens = word_tokenize(sentence)
#  X_test_len = len(tokens)
#  X_test = prepareData(tokens, vocab, max_len)
#  X_test = torch.LongTensor(X_test).cuda()
#  X_test_len = torch.FloatTensor(X_test_len).cuda()
#  output = model(X_test, X_test_len)
#  prediction = output.max(dim=1)[1]
#  print(prediction)

In [0]:
# Cellule pour éviter les doublons dans les positifs et négatifs.

files_pos = [f for f in listdir("/content/gdrive/My Drive/datas/pixabay/dogs/1")]
files_neg = [f for f in listdir("/content/gdrive/My Drive/datas/pixabay/dogs/0")]

cnt = 0
for f in files_pos:
  if f in files_neg:
    print("{} in Negatives.".format(f))
    cnt += 1
  else:
    print("{} not in Negatives".format(f))
    
print(cnt)

Dans ce cadre précis il faut ajouter du padding à chaque image. Pour se faire, nous devons rechercher la largeur max et la longueur max.

In [0]:
class AddPadding:
  def __init__(self, maxHeight, maxWidth):
    self.maxHeight = maxHeight
    self.maxWidth = maxWidth

  def __call__(self, img):
    # Utilisation du code https://discuss.pytorch.org/t/add-padding-to-images/24309/3
    delta_width = self.maxWidth - img.size[0]
    delta_height = self.maxHeight - img.size[1]
    pad_width = delta_width //2
    pad_height = delta_height //2
    padding = (pad_width,pad_height,delta_width-pad_width,delta_height-pad_height)
    return ImageOps.expand(img, padding)

In [28]:
# Importation des images pour avoir les dimensions max
# TODO : Améliorer ce passage de code avec du multi-threading (ralentissement important à partir de 60%)
img_path = "/content/gdrive/My Drive/datas/pixabay/dogs/"
files = glob(img_path + "1/*")
maxH = 0
maxW = 0
total = len(files)
cnt = 0
for f in files:
  img = Image.open(f)
  w = img.width
  h = img.height
  if w > maxW:
    maxW = w
  if h > maxH:
    maxH = h
  cnt += 1
  print("\rAvancement : {:.2f}%".format(cnt/total*100), end='')

files = None
print("\nMax height : {}\nMax width : {}".format(maxH, maxW))

Avancement : 100.00%Max height : 340
Max width : 1093


In [0]:
# On charge les dataloaders (en supprimant les images précédentes pour libérer l'espace)
all_transforms = transforms.Compose(
    [
     AddPadding(maxH, maxW),
     transforms.ToTensor()
    ]
)
train_dataset = torchvision.datasets.ImageFolder(
    root=img_path,
    transform=all_transforms
)

In [0]:
def create_dataloader(dataset):
    train_loader = DataLoader(
        dataset,
        batch_size=128,
        num_workers=0,
        shuffle=True
    )
    return train_loader

data_loader = create_dataloader(train_dataset)

model = torchvision.models.resnet18(pretrained=True)

# La classification se fait sur 2 classes uniquement
model.fc = nn.Linear(512, 2)
print(model)

In [0]:
training(model, data_loader)